In [52]:
import json
import glob

from transformers import AutoTokenizer
from datasets import *

def load_json_data(folder_path, key):
    data = []
    for file_path in glob.glob(folder_path + '/*.json'):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            data.extend(json_data[key])
    return data

raw_pages = load_json_data("../data_prep/data", key="ocr_results")
cleaned_pages = load_json_data("../data_prep/data", key="cleaned_pages")

test_size = 0.2
train_size = int(len(cleaned_pages) * (1 - test_size))
train_raw_pages, test_raw_pages = raw_pages[:train_size], raw_pages[train_size:]
train_cleaned_pages, test_cleaned_pages = cleaned_pages[:train_size], cleaned_pages[train_size:]

dataset = DatasetDict({
    'train': Dataset.from_dict({"raw_pages": train_raw_pages[:len(train_cleaned_pages)], "cleaned_pages": train_cleaned_pages}),
    'test': Dataset.from_dict({"raw_pages": test_raw_pages[:len(test_cleaned_pages)], "cleaned_pages": test_cleaned_pages})
})

base_model = "unsloth/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

instruction = """
Du bist ein Experte für Textbereinigung. Deine Aufgabe ist es, einen Eingabetext zu bereinigen, der aus einem PDF-Dokument extrahiert wurde. Der Inhalt ist immer nur von einer einzelnen Seite, es sollte also nicht zu viel Text auf einmal sein. Es ist sehr wichtig, dass keine Daten und Informationen verloren gehen und dass die Originaltexte in keiner Weise verändert werden!
Antworte ausschließlich in Deutsch und keiner anderen Sprache.

Du hast die folgenden Aufgaben:
- Entferne alle seltsamen Textteile und Sonderzeichen.
- Entferne alle unnötigen Leerzeichen und Zeilenumbrüche.
- Organisiere die Formatierung.
- Korrektur von Rechtschreibfehlern.
- Handling von Formatierungsfehlern.

Gib nur den bereinigten und formatierten Text zurück und nichts anderes! Füge keinen eigenen Text hinzu! Achte auf Vollständigkeit, es darf kein Inhalt verloren gehen und es muss alles 100 % vollständig sein!
"""

def format_chat_template(row):
    
    row_json = [{"role": "system", "content": instruction},
               {"role": "user", "content": row["raw_pages"]},
               {"role": "assistant", "content": row["cleaned_pages"]}]
    
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template
)